In [ ]:
#base imports
import os
import sys
import shutil

import numpy as np
import pandas as pd

import enlighten

from IPython.core.display import HTML

import torch
import torch.nn.functional as F
from torch.nn import Sequential, Linear, ReLU, GRU
import torch_geometric.transforms as T
from torch_sparse import coalesce
from torch_geometric.data import (InMemoryDataset, Data, download_url)
from torch_scatter import scatter

from torch_geometric.nn import NNConv, Set2Set
# from torch_geometric.data import DataLoader
from torch_geometric.loader import DataLoader
from torch_geometric.utils import remove_self_loops

In [ ]:
#RDKit
import rdkit
from rdkit import Chem
from rdkit.Chem.rdchem import HybridizationType
from rdkit.Chem.rdchem import BondType as BT
from rdkit.Chem import PandasTools
from rdkit import RDLogger

RDLogger.logger().setLevel(RDLogger.CRITICAL)

In [ ]:
#fcts
def show(df):
    """Render the molecules within a DataFrame correctly"""
    return HTML(df.to_html(notebook=True))

In [ ]:
assay = 1511
global dataset
# filename = f'{assay}_dataset.pt'
dataset = f'{assay}_dataset.pt'
global input_sdf
input_sdf = f'{assay}_compounds.sdf'
global directory
directory = f'{assay}_data'

In [ ]:
#setup dataset
sdfpath = os.path.join(os.getcwd(), f'{assay}_compounds.sdf')
backup_df = PandasTools.LoadSDF(sdfpath)
dataframe = backup_df.copy()

In [ ]:
show(dataframe.head(5))

In [ ]:
import csv

global element_dict
elements = []

if os.path.exists('element_list.json'):
    pass
else:
    with open('periodictable.csv', 'r') as ifile:
        reader = csv.reader(ifile)
        for i in reader:
            element = i[1]
            elements.append(element)

    indexes = [i for i in range(len(elements))]


    element_dict = dict(zip(elements, indexes))

In [ ]:
import json

if os.path.exists('element_list.json'):
    with open('element_list.json', 'r') as listfile:
        element_dict = json.load(listfile)
else:
    with open('element_list.json', 'w') as listfile:
        json.dump(element_dict, listfile)

In [ ]:
#setup filesystem
import shutil

rawdir = os.path.join(directory, 'raw')
processdir = os.path.join(directory, 'processed')

try:
    os.mkdir(directory)
    # rawdir = os.path.join(directory, 'raw')
    # processdir = os.path.join(directory, 'processed')
    os.mkdir(rawdir)
    os.mkdir(processdir)
except FileExistsError:
    print("Directory already exists")
    pass
finally:
    shutil.copyfile(os.path.abspath(input_sdf), os.path.join(rawdir, input_sdf))

In [ ]:
dataframe["PUBCHEM_ACTIVITY_OUTCOME"].replace("Inactive", 0, inplace=True)
dataframe["PUBCHEM_ACTIVITY_OUTCOME"].replace("Active", 1, inplace=True)

In [ ]:
dataframe.PUBCHEM_ACTIVITY_OUTCOME.value_counts()

In [ ]:
global targets
targets = dataframe.PUBCHEM_ACTIVITY_OUTCOME.to_list()

In [ ]:
print(torch.version.cuda)

In [ ]:
class BioAssayDataset(InMemoryDataset):
    def __init__(self, root=None, transform=None, pre_transform=None):
        super().__init__(root, transform, pre_transform)
        self.data, self.slices = torch.load(self.processed_paths[0])

    @property
    def raw_file_names(self):
        return [input_sdf]
    
    @property
    def processed_file_names(self):
        return [dataset]
    
    def download(self):
        pass

    def process(self):
        data_list = []

        bonds = {BT.SINGLE: 0, BT.DOUBLE: 1, BT.TRIPLE: 2, BT.AROMATIC: 3}
        types = element_dict


        # mol_list = Chem.SDMolSupplier(self.raw_paths[0], removeHs=False, sanitize=False)
        mol_list = Chem.SDMolSupplier(self.raw_paths[0], removeHs=False)
        pbar = enlighten.Counter(total=len(mol_list), desc='Encoding molecules...', unit='ticks')
        for ID, mol in enumerate(mol_list):
            #number of atoms
            n_a = mol.GetNumAtoms()


            conf = mol.GetConformer()
            pos = conf.GetPositions()
            pos = torch.tensor(pos, dtype=torch.float)

            #atom features
            type_idx = []
            atomic_number = []
            aromatic = []
            sp = []
            sp2 = []
            sp3 = []
            num_hs = []
        
            for atom in mol.GetAtoms():
                #need to implement this
                type_idx.append(types[atom.GetSymbol()])

                atomic_number.append(atom.GetAtomicNum())
                aromatic.append(1 if atom.GetIsAromatic() else 0)
                hybridisation = atom.GetHybridization()
                sp.append(1 if hybridisation == HybridizationType.SP else 0)
                sp2.append(1 if hybridisation == HybridizationType.SP2 else 0)
                sp3.append(1 if hybridisation == HybridizationType.SP3 else 0)
                num_hs.append(atom.GetTotalNumHs(includeNeighbors=True))

            z = torch.tensor(atomic_number, dtype=torch.long)            

            #bonds
            row, col, edge_type = [], [], []
            for bond in mol.GetBonds():
                start, end = bond.GetBeginAtomIdx(), bond.GetEndAtomIdx()
                row += [start, end]
                col += [end, start]
                edge_type += 2 * [bonds[bond.GetBondType()]]

            edge_index = torch.tensor([row, col], dtype=torch.long)
            edge_type = torch.tensor(edge_type, dtype=torch.long)
            edge_attr = F.one_hot(edge_type,num_classes=len(bonds)).to(torch.float)

            # perm = (edge_index[0] * n_a + edge_index[1]).argsort()

            # # print(edge_index[0])
            # # print(edge_index[0])
            # # print(n_a)
            # edge_index = edge_index[:, perm]
            # edge_type = edge_type[perm]
            # edge_attr = edge_attr[perm]

            # row, col = edge_index
            # hs = (z==1).to(torch.float)
            # num_hs = scatter(hs[row], col, dim_size=n_a).tolist()

            x1 = F.one_hot(torch.tensor(type_idx), num_classes=len(types))
            x2 = torch.tensor([atomic_number, aromatic, sp, sp2, sp3, num_hs], dtype=torch.float).t().contiguous()
            x = torch.cat([x1.to(torch.float), x2], dim=-1)

            #target is the binarised activities
            y = targets[ID]

            name = mol.GetProp('_Name')

            data = Data(x=x, z=z, pos=pos, edge_index=edge_index, edge_attr=edge_attr, y=y, name=name, idx=ID)
            data_list.append(data)
            pbar.update()
        torch.save(self.collate(data_list), self.processed_paths[0])

    #need to add process functions for the molecules

# class MyTransfrom(object):
#     def __call__(self, data):
#         print(data)
#         data.y = data.y[:, targets]
#         print(data.y)
#         return data
        
class Complete(object):
    def __call__(self, data):
        device = data.edge_index.device

        row = torch.arange(data.num_nodes, dtype=torch.long, device=device)
        col = torch.arange(data.num_nodes, dtype=torch.long, device=device)

        row = row.view(-1, 1).repeat(1, data.num_nodes).view(-1)
        col = col.repeat(data.num_nodes)
        edge_index = torch.stack([row, col], dim=0)

        edge_attr = None
        if data.edge_attr is not None:
            idx = data.edge_index[0] * data.num_nodes + data.edge_index[1]
            size = list(data.edge_attr.size())
            size[0] = data.num_nodes * data.num_nodes
            edge_attr = data.edge_attr.new_zeros(size)
            edge_attr[idx] = data.edge_attr

        edge_index, edge_attr = remove_self_loops(edge_index, edge_attr)
        data.edge_attr = edge_attr
        data.edge_index = edge_index

        return data


In [ ]:
transform = T.Compose([Complete(), T.Distance(norm=False)])

dataset = BioAssayDataset(directory, transform=transform).shuffle()

In [ ]:
dataset.data.y.view(1,-1).shape

In [ ]:
for x in dataset:
    print(x)
    break

In [ ]:
#split datasets into train/test

td_len = len(dataset)
tr_part = round(td_len*0.8)
val_part = round(td_len*0.1)

test_set = dataset[(tr_part+val_part):]
val_set = dataset[tr_part:(tr_part+val_part)]
train_set = dataset[:tr_part]

train_loader = DataLoader(test_set, batch_size=128, shuffle=False)
test_loader = DataLoader(test_set, batch_size=2160, shuffle=False)
val_loader = DataLoader(val_set, batch_size=128, shuffle=False)

In [ ]:
dim = 64

In [ ]:
#setup torch NN

class Net(torch.nn.Module):
    def __init__(self):
        super(Net,self).__init__()
        self.lin0 = torch.nn.Linear(dataset.num_features, dim)

        nn = Sequential(Linear(5,128), ReLU(), Linear(128, dim * dim))
        self.conv = NNConv(dim, dim, nn, aggr='mean')
        self.gru = GRU(dim, dim)

        self.set2set = Set2Set(dim, processing_steps=3)
        self.lin1 = torch.nn.Linear(2 * dim, dim)
        self.lin2 = torch.nn.Linear(dim, 1)


    def forward(self, data):
        out = F.relu(self.lin0(data.x))
        h = out.unsqueeze(0)

        for i in range(3):
            m = F.relu(self.conv(out, data.edge_index, data.edge_attr))
            out, h = self.gru(m.unsqueeze(0), h)
            out = out.squeeze(0)

        out = self.set2set(out, data.batch)
        out = F.relu(self.lin1(out))
        out = self.lin2(out)

        return out.view(-1)



In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
model = Net().to(device)

In [ ]:
optimiser = torch.optim.Adam(model.parameters(), lr=0.001)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimiser, mode='min',
                                                        factor=0.7, patience=5,
                                                        min_lr=0.00001)

In [ ]:
def train(epoch):
    model.train()
    loss_all = 0

    for data in train_loader:
        data = data.to(device)
        optimiser.zero_grad()

        loss = F.mse_loss(model(data), data.y)
        # print(loss)
        loss.backward()
        loss_all += loss.item() * data.num_graphs
        optimiser.step()

    return loss_all / len(train_loader.dataset)


def test(loader):
    model.eval()
    error = 0

    for data in loader:
        data = data.to(device)
        error += (model(data) - data.y).abs().sum().item()
        return error / len(loader.dataset)

In [ ]:
max_epochs = 300

In [ ]:
best_val_error = None

for epoch in range(1, max_epochs):
    lr = scheduler.optimizer.param_groups[0]['lr']
    loss = train(epoch)
    val_error = test(val_loader)
    scheduler.step(val_error)

    if best_val_error is None or val_error <= best_val_error:
        test_error = test(test_loader)
        best_val_error = val_error

    print(f' Epoch {epoch}, LR: {lr}, Loss: {loss}, Validation MAE: {val_error}, Test MAE: {test_error}')